In [ ]:
!pip install line-bot-sdk flask flask-ngrok 

# 安裝後必須點擊Runtime -> Restart Runtime
!pip install  google-cloud-texttospeech


In [ ]:
'''
引用套件
'''

# 引用Web Server套件
from flask import Flask, request, abort, jsonify

# 載入json處理套件
import json

# 外部連結自動生成套件
from flask_ngrok import run_with_ngrok

# 從linebot 套件包裡引用 LineBotApi 與 WebhookHandler 類別
from linebot import (
    LineBotApi, WebhookHandler
)

# 引用無效簽章錯誤
from linebot.exceptions import (
    InvalidSignatureError
)


In [ ]:
'''
建置主程序

建置handler與 line_bot_api
'''

# 設定Server啟用細節
app = Flask(__name__,static_url_path = "/material" , static_folder = "./material/")
run_with_ngrok(app)

# 生成實體物件
line_bot_api = LineBotApi("CHANNEL_ACCESS_TOKEN")
handler = WebhookHandler("CHANNEL_SECRET")
  
from google.cloud import storage
from google.cloud import texttospeech
# 本地端才如此作為，最好改用環境變數
storage_client = storage.Client.from_service_account_json('service_account.json')
bucket_name='GCP Cloud Storage Bucket name'

# Instantiates a client
speech_client = texttospeech.TextToSpeechClient.from_service_account_json('service_account.json')

In [ ]:
'''
建置主程序的API入口
  接受Line傳過來的消息
  並取出消息內容
  將消息內容存在google drive的檔案內
  並請handler 進行消息驗證與轉發
'''

# 啟動server對外接口，使Line能丟消息進來
@app.route("/", methods=['POST'])
def callback():
    # get X-Line-Signature header value
    signature = request.headers['X-Line-Signature']

    # get request body as text
    body = request.get_data(as_text=True)
    print(body)

    # 記錄用戶log，請去查更正確的logging作法
    f = open("./ai-event.log", "a")
    f.write(body)
    f.close()

    # handle webhook body
    try:
        handler.handle(body, signature)
    except InvalidSignatureError:
        abort(400)

    return 'OK'

In [ ]:
'''

若收到圖片消息時，

先回覆用戶文字消息，並從Line上將照片拿回。

'''

from linebot.models import(
    MessageEvent,ImageMessage, TextSendMessage
)

@handler.add(MessageEvent, message=ImageMessage)
def handle_message(event):
    
    line_bot_api.reply_message(
        event.reply_token,
        TextSendMessage(text='Image has Upload'+ ' ' + event.message.id))
    
    message_content = line_bot_api.get_message_content(event.message.id)
    with open(event.message.id+'.jpg', 'wb') as fd:
        for chunk in message_content.iter_content():
            fd.write(chunk)


    source_file_name= event.message.id+'.jpg'
    destination_blob_name=  event.source.user_id +'/'+ event.message.id+'.jpg'

    bucket = storage_client.bucket(bucket_name)

    blob = bucket.blob(destination_blob_name)

    blob.upload_from_filename(source_file_name)

In [ ]:
'''
若收到文字消息時

將文字傳給GCP，轉譯成音訊後

放到cloud storage，並組合成AudioSendMessage，傳回給用戶

Podcast
'''

from linebot.models import(
    MessageEvent,TextMessage, AudioSendMessage
)


@handler.add(MessageEvent, message=TextMessage)
def handle_text_message(event):

  # Set the text input to be synthesized
  synthesis_input = texttospeech.SynthesisInput(text=event.message.text)


  # Build the voice request, select the language code ("en-US") and the ssml
  # voice gender ("neutral")
  voice = texttospeech.VoiceSelectionParams(
      language_code="zh-TW", ssml_gender=texttospeech.SsmlVoiceGender.NEUTRAL
  )

  # Select the type of audio file you want returned
  audio_config = texttospeech.AudioConfig(
      audio_encoding=texttospeech.AudioEncoding.MP3
  )

  # Perform the text-to-speech request on the text input with the selected
  # voice parameters and audio file type
  response = speech_client.synthesize_speech(
    input=synthesis_input, voice=voice, audio_config=audio_config
  )

  # The response's audio_content is binary.
  voice_mp3_file_path=event.message.id+".mp3"
  with open(voice_mp3_file_path, "wb") as out:
      # Write the response to the output file.
      out.write(response.audio_content)
      print('Audio content written to file '+voice_mp3_file_path)


  # 上傳檔案
  destination_blob_name=  event.source.user_id +'/'+ event.message.id+'.mp3'

  bucket = storage_client.bucket(bucket_name)

  blob = bucket.blob(destination_blob_name)

  blob.upload_from_filename(voice_mp3_file_path)

  # 回話
  line_bot_api.reply_message(
      event.reply_token,
      AudioSendMessage(
       original_content_url ="https://storage.googleapis.com/"+bucket_name+'/'+destination_blob_name,
       duration=60000
      ) 
  )

In [ ]:
'''
Google ML 參考
https://colab.research.google.com/github/robeartoe/APIWorkshop/blob/master/MLWorkshop.ipynb#scrollTo=Snq8ehAu7JkD

'''

In [ ]:
# 主程序運行
app.run()